In [1]:
import math
import tsv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
root_path = "../data/poly_mo/"

# preprocess data files

# Quant_bootstraps.tsv :containing the matrix of bootstrap experiments
# containing the final count for each transcript in each round of bootstrapping
# with a row be a bootstrap output and columns be list of transcripts.
quant_bootstraps = tsv.TsvReader(open(root_path+"quant_bootstraps.tsv"))
count = 0
quant_boot = []
for parts in quant_bootstraps:
    quant_boot.append(parts)
df_quant_boot = pd.DataFrame.from_records(quant_boot[1:], columns=quant_boot[0])
df_quant_boot = df_quant_boot.astype('float')
df_quant_boot_mean = df_quant_boot.mean()
df_quant_boot_std = df_quant_boot.std()
id_in_quant_boot = list(df_quant_boot.columns)

# Quant.sf :estimated quantifications for each transcript
quant_file = open(root_path+"quant.sf")
lines = quant_file.readlines()
quant_file.close()
count = 0
quant = []
for line in lines:
    line = line[:-1]
    l = line.split('\t')
    quant.append(l)

df_quant = pd.DataFrame.from_records(quant[1:], columns=quant[0])
df_quant.Name = df_quant.Name.astype(str)
df_quant.Length = df_quant.Length.astype(int)
df_quant.EffectiveLength = df_quant.EffectiveLength.astype(float)
df_quant.TPM = df_quant.TPM.astype(float)
df_quant.NumReads = df_quant.NumReads.astype(float)

# eq_classes

# we now have:
# df_quant_boot_mean
# df_quant_boot_std
# df_quant.Name
# df_quant.Length
# df_quant.EffectiveLength
# df_quant.TPM
# df_quant.NumReads

fixed_dfquant_boot = df_quant_boot.copy()
columns = fixed_dfquant_boot.columns

sort_qb = []
for id in columns:
    try:
        listed = list(df_quant_boot[id])
    except KeyError:
        pass
    else:
        listed.sort()
        sort_qb.append(listed)
sort_qb = list(map(list,zip(*sort_qb)))

df_qb_sorted = pd.DataFrame.from_records(sort_qb, columns=columns)

sum = len(sort_qb)
print(int(sum*0.025))
print(int(sum*0.975))
percent2dot5 = df_qb_sorted.loc[int(sum*0.025)-1]
percent97dot5 = df_qb_sorted.loc[int(sum*0.975)-1]

remove_ids = []
for id in columns:
    if float(percent97dot5[id]) == 0:
        remove_ids.append(id)



5
195


In [ ]:
new_columns = set(columns) - set(remove_ids)
new_columns = list(new_columns)
fixed_df_quant_boot = fixed_df_quant_boot[new_columns]

In [5]:
fixed_df_quant_boot = fixed_df_quant_boot

In [ ]:
df_quant_boot_std = fixed_df_quant_boot.std()
df_quant_boot_mean = fixed_df_quant_boot.mean()

In [6]:
df_quant.describe()

,Length,EffectiveLength,TPM,NumReads
count,93109.000000,93109.000000,93109.000000,9.310900e+04
mean,2008.924164,1810.144491,10.740100,3.579084e+02
std,2132.499138,2132.174682,201.478857,6.924434e+03
min,21.000000,9.784000,0.000000,0.000000e+00
25%,640.000000,441.002000,0.000000,0.000000e+00
50%,1344.000000,1144.991000,0.000000,0.000000e+00
75%,2640.000000,2439.991000,0.173865,6.000000e+00
max,109224.000000,109024.991000,23356.420222,1.109005e+06


In [8]:
df_quant.head(1)

,Name,Length,EffectiveLength,TPM,NumReads
0,ENST00000382410,587,388.005,0.0,0.0


In [14]:
new_column = list(map(list,zip(*new_columns)))

In [26]:
label = []
for i in range(len(new_columns)):
    label.append(0)

In [28]:
nc = [new_columns,label]
nc = list(map(list,zip(*nc)))

In [29]:
nc[0:10]

[['ENST00000336614', 0],
 ['ENST00000409777', 0],
 ['ENST00000370760', 0],
 ['ENST00000607698', 0],
 ['ENST00000328600', 0],
 ['ENST00000255695', 0],
 ['ENST00000491222', 0],
 ['ENST00000560778', 0],
 ['ENST00000367419', 0],
 ['ENST00000607296', 0]]

In [30]:
new_col = pd.DataFrame.from_records(nc, columns=['Name','useless'])

In [31]:
new_col.head(10)

,Name,useless
0,ENST00000336614,0
1,ENST00000409777,0
2,ENST00000370760,0
3,ENST00000607698,0
4,ENST00000328600,0
5,ENST00000255695,0
6,ENST00000491222,0
7,ENST00000560778,0
8,ENST00000367419,0
9,ENST00000607296,0


In [36]:
fix_df_quant = new_col.merge(df_quant, on='Name')

In [37]:
fix_df_quant.describe()

,useless,Length,EffectiveLength,TPM,NumReads
count,33965.0,33965.000000,33965.000000,33965.000000,3.396500e+04
mean,0.0,2262.225438,2063.146050,29.442073,9.811421e+02
std,0.0,2381.743492,2381.672015,332.764064,1.143815e+04
min,0.0,82.000000,9.804000,0.000000,0.000000e+00
25%,0.0,755.000000,555.999000,0.085157,2.939223e+00
50%,0.0,1628.000000,1428.991000,1.237854,4.700000e+01
75%,0.0,2946.000000,2746.991000,8.238800,3.988340e+02
max,0.0,104301.000000,104101.991000,23356.420222,1.109005e+06


In [38]:
useless = fix_df_quant.pop('useless')

In [39]:
fix_df_quant.describe()

,Length,EffectiveLength,TPM,NumReads
count,33965.000000,33965.000000,33965.000000,3.396500e+04
mean,2262.225438,2063.146050,29.442073,9.811421e+02
std,2381.743492,2381.672015,332.764064,1.143815e+04
min,82.000000,9.804000,0.000000,0.000000e+00
25%,755.000000,555.999000,0.085157,2.939223e+00
50%,1628.000000,1428.991000,1.237854,4.700000e+01
75%,2946.000000,2746.991000,8.238800,3.988340e+02
max,104301.000000,104101.991000,23356.420222,1.109005e+06
